<a href="https://colab.research.google.com/github/CE339/CES-S24/blob/main/Module04/MinCostFlow_supersource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initializing Google Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [18]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

In [ ]:
!pip install pyomo
!apt-get install -y -qq coinor-cbc  #!apt-get install -y -qq glpk-utils

**Importing Pyomo and CBC Solver**

In [4]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('cbc',executable='/usr/bin/cbc')

**Building the abstract model**

We will use an **abstract model** representation to the min cost network flow problem.\
As mentioned in one previous example, **abstract models** are just defined by symbols and rules.

The idea is to create a general model that would be used to solve multiple **instances** fo the same optimization problem. In other words we could solve the same problem for different parameters.

In [27]:
from math import inf

model = AbstractModel()

model.N = Set() #Nodes set
model.A = Set(within=model.N*model.N) #Arcs set

# ci - Shipping costs
model.supply = Param(model.N, within=Reals)
model.c      = Param(model.A, within=Reals) #Changed from positive reals with respect to the min cost network flow problem
model.ub     = Param(model.A, within=NonNegativeReals)
#!!!New parameter to enforce lower bound on each arc
model.lb     = Param(model.A, within=NonNegativeReals)

# Flow in the arc defined as decision variable xij
model.x  = Var(model.A,   within=NonNegativeReals)

# Minimize the flow cost in the network
def cost_rule(model):
    return sum(model.c[i,j]*model.x[i,j] for (i, j) in model.A)
model.total = Objective(rule=cost_rule, sense=minimize)

# Enforce an upper limit on the flow across each arc
def limit_rule(model, i, j):
    return model.x[i,j] <= model.ub[i,j]
model.limit_ub = Constraint(model.A, rule=limit_rule)

# !!!New constraint to enforce a lower limit on the flow across each arc
def limit_rule(model, i, j):
    return model.x[i,j] >= model.lb[i,j]
model.limit_lb = Constraint(model.A, rule=limit_rule)

# Enforce flow through each node
def flow_rule(model, k):
    inFlow  = sum(model.x[i,j] for (i,j) in model.A if j == k)
    outFlow = sum(model.x[i,j] for (i,j) in model.A if i == k)
    return inFlow + model.supply[k] == outFlow
model.flow = Constraint(model.N, rule=flow_rule)

#------------------------------------------------------------------------------
#Suffix for dual variables reporting. See more details at https://pyomo.readthedocs.io/en/stable/working_models.html
#------------------------------------------------------------------------------
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
model.rc = Suffix(direction=Suffix.IMPORT_EXPORT)

**Creating the model instance** - using data contained in the data file

In [ ]:
instance = model.create_instance('netflow_supersource.dat')
instance.pprint()

**Optimizing the model instance**

In [ ]:
results = opt.solve(instance, tee=True)

**Display Results**

In [ ]:
print(instance.display())

**Looking at the Dual Variables Associated with the Constraints**

In [ ]:
instance.dual.pprint() #Printing Dual Vars
instance.rc.pprint() #Printing Reduced Costs